In [1]:
# Initial Jupyter Script

In [ ]:
## imports

import requests
from requests.auth import HTTPBasicAuth
import json
import uuid
import configparser

header_correlation_id = 'x-correlation-id'
header_content_type = 'Content-Type'
header_client = 'x-client-id'
header_host = 'host'

In [ ]:
## Generate token
def create_token (environment):
    token_url =  'https://wills-orchestrator.' + environment + '.private.strange.com:443'

    token_headers = {
        header_correlation_id: str( uuid.uuid4() ),
        header_content_type: 'application/json',
        header_host: 'https://wills-orchestrator.' + environment + '.private.strange.com'
    }

    # submit token request
    token_response = requests.get(token_url, header=token_headers, verify=False)

    # parse token
    payload = token_response.json()
    return "Bearer " + payload['token']

In [ ]:
## Query for order
def query_order_request(environment , client):
    base_url = 'https://wills-orchestrator.' + environment + '.private.strange.com:443'
    query_url = '/ws/refund/cs/query'

    headers = {
        header_correlation_id: str( uuid.uuid4() ),
        header_content_type: 'application/json',
        header_client: client
    }

    url = base_url + query_url
    response = requests.post( url , headers=headers , verify=False )
    response_json = response.json()
    print( 'Query Response: '  + json.dumps(response_json, indent=2 ))


In [3]:
def xml_alert(environment , merchant , client , dispute):
    base_url = 'https://wills-orchestrator.' + environment + '.private.strange.com:443'
    alert_url = '/ws/alerts'

    headers = {
        header_correlation_id: str( uuid.uuid4() ),
        header_content_type: 'application/xml',
        header_client: client
    }

    if dispute == True:
        alert_type = 'ConfirmedDispute'
    else:
        alert_type = 'CustomerFraud'

    data = f"""
    <?xml version="1.0" encoding="UTF-8" standalone="yes"?>
    <AlertNotification>
        <{alert_type}>
        <Alert>
            <AlertID>354J434N11K4545Nl</alertID>
            <AlertTimestamp>2025-01-22T01:01:01.000Z</alertTimestamp>
            <CardNumber>421124xxxxxx1234</CardNumber>
            <ARN>24568923450901234324</ARN>
            <Amount>12.34</Amount>
            <Currency>USD</Currency>
            <TransactionType>eCommerce</TransactionType>
            <AuthCode>AUTH1</AuthCode>
            <MerchantID>{merchant}</MerchantID>
        </Alert>
        </{alert_type}>
    </AlertNotification>
    """

    alert_response = requests.post(url=base_url+alert_url, data=data, headers=headers, verify=False)

    print(alert_response)

In [ ]:
def create_inquiry_token (environment):
    token_url =  'https://wills-orchestrator.' + environment + '.private.strange.com:443'

    token_headers = {
        header_correlation_id: str( uuid.uuid4() ),
        header_content_type: 'application/json',
        header_host: 'https://wills-orchestrator.' + environment + '.private.strange.com'
    }

    # submit token request
    token_response = requests.get(token_url, header=token_headers, verify=False)

    # parse token
    token_response_token = token_response.json()
    return "Bearer " + token_response_token['token']